In [176]:
import sys
import re

In [ ]:
#Step 1. Get the list of words with i and ї from a sample corpus

def WordsWithI():
    
    file = open('sample_corpus.txt', 'r', encoding='utf-8')
    #words = re.findall('[^\s]*i(?!werty)[^\s]*', file) #[^\s]*q(?!werty)[^\s]*
    words = re.findall('ˆ|\s([a-zA-Zа-яА-ЯіїЇєЄґҐ]*i|I|і|ї[a-zа-яіїєґ]*)[.?,!"()]|\s|$', file) 
    
    f = open('words with i', 'w', encoding='utf-8')
    
    for word in words:
        f.write('%s\t\n'% (word))
    f.close()
    file.close()
    
WordsWithI()

#Step 2. Handwork - put the equivalents in Russian and rename it to 'project_lexics_2'

In [119]:
#Step 3. Make an aestetic work, using HW 2-4 (see 1-3 in goals). 

#Dictionaries Maker from hand-based analytics of the sample corpus.

#Dictionary 1 with letters, morphemes, phonetic features to change:

def Dictionaries():
    letters_phon_morph = open('project_letters.txt', 'r', encoding='utf-8')
    let_ph_mo = {}
    for line in letters_phon_morph:
        line = line.replace('\n', '')
        line = line.split('\t')
        let_ph_mo[line[0]] = line[1]
    
#Dictionary 2 with short lexems:    
    lexics = open('project_lexics.txt', 'r', encoding='utf-8')
    lexic = {}
    for line in lexics:
        line = line.replace('\n', '')
        line = line.split('\t')
        lexic[line[0]] = line[1]

#Dictionary 3 based on Step 1:  
    lexics_2 = open('project_lexics_2.txt', 'r', encoding='utf-8')
    lexic2 = {}
    for line in lexics_2:
        line = line.replace('\n', '')
        line = line.split('\t')
        lexic2[line[0]] = line[1]
        
Dictionaries()

In [121]:
#'Cosmetic' code: changes phonetical, morhological and small lexical features

def Step3():
    
    file = open('sample_corpus.txt', 'r', encoding='utf-8') #actually we can use wiki.txt here
    f = open('cosmetic_wiki.txt', 'w', encoding='utf-8')

    Dictionaries()
    for word in file:
        if word.strip() == '': continue
        for key, value in lexic.items():
            word = re.sub(key, value, word) #я хочу, чтобы менялось только полностью слово, а часть слова не была затронута
        #for key, value in lexic2.items():
            #project_lexics_2.txtword = re.sub(key, value, word)
        word = word.strip().split(' ')
        for w in word:
            if w.strip() == '': continue
            tr = w
            for key, value in let_ph_mo.items():
                tr = tr.replace(key, value)

            f.write((tr) + ' ')
        #print()
    file.close()
    f.close()
    
Step3()


In [181]:
#Step 4. Train part. HW 4

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

#Преобразовать слово внутри списка в список, запомнить букву в списке по индексу и на этом же месте найти соответсвующую русскую букву
#Ненужные буквы удалить, чтобы осталось только руссинские i и их русские эквиваленты
#На выходе получится файл

def Preparation():
    
    lines = open('project_lexics_2.txt', 'r', encoding='utf-8')
    f = open('train.txt', 'w', encoding='utf-8')
    letters = []
    
    for line in lines:
        if '\t' not in line:
            continue
        line = re.sub('\n', '', line)
        row = line.split('\t')
        for r in row:
            if r == '':
                row.remove(r)
            else:
                break
            for letter in r:
                letters.append(letter)
            
            print(letters)
            
    lines.close()
    f.close()

Preparation()

In [185]:
#adobted spectei's codes

def Train():
    
    # read all of the input lines in our trained meanings
    lines = open('train.txt', 'r', encoding='utf-8')
    f = open('project_model.tsv', 'w', encoding='utf-8')

    meaning_count = {}
    letters = {}
    letters_of_letter = {}
    total = 0

    for line in lines:
        if '\t' not in line:
            continue
        row = line.split('\t')
        
        meaning = row[1]

        # if we haven't seen the meaning before then initialise the count to 0
        if meaning not in meaning_count:
            meaning_count[meaning] = 0
        meaning_count[meaning] = meaning_count[meaning] + 1

        letter = row[0]
        
        if letter not in letters_of_letter: 
            letters_of_letter[letter] = 0
        letters_of_letter[letter] = letters_of_letter[letter] + 1
        
        if letter not in letters:
            letters[letter] = {}

        if meaning not in letters[letter]:
            letters[letter][meaning] = 0
        letters[letter][meaning] = letters[letter][meaning] + 1
        total = total + 1
        #print(meaning_count[meaning], meaning, row)
    # {'ADJ': 2, 'NOUN': 5, 'NUM': 2, '_': 80, 'PROPN': 3}                                                       

    for meaning in meaning_count:
        freq = meaning_count[meaning]/total
        #print('%.2f\t%d\t%s\t%s' % (freq, meaning_count[meaning], meaning, '_'))

    # for each of the words in the matrix
    for letter in letters:
        # for each of the tag
        for meaning in letters[letter]:
            freq = letters[letter][meaning]/letters_of_letter[letter]
            f.write('%.2f\t%d\t%s\t%s' % (freq, letters[letter][meaning], meaning, letter))
            print('%.2f\t%d\t%s\t%s' % (freq, letters[letter][meaning], meaning, letter))


    f.close()        
    lines.close()
Train()

1.00	3	i
	i
0.67	4	и
	і
0.17	1	о
	і
0.17	1	е
	і
0.67	2	е
	ї
0.33	1	е	ї


In [ ]:
#Step 5. Test. HW 5

#spectei's codes

def Test():

    fd = open('project_model.tsv', 'r', encoding='utf-8')


    frequent_tag = 'X' # variable to store the most frequent tag we have seen
    last_max = 0 # variable to store the maximum frequency we have seen

    word_tag = {}  # word_tag['cat'] = 'NOUN'
    word_prob = {} # word_prob['cat'] = 0.6

    for line in fd:
        line = line.strip() # take away newline characters at the beginning/end of the line
        line = line.split('\t') # split the line on tab to make a list X 
        # ['0.17', '3', 'PROPN', '_']
        if int(line[1]) > last_max: # if the frequency of the tag is higher than the previous maximum
            last_max = int(line[1]) # set the new maximum	
            frequent_tag = line[2]

        # if we have never seen the surface form before, the most frequent tag is the one we just saw
        surface_form = line[3] # this is the surface form
        prob = float(line[0]) # this is the probability
        tag = line[2] # this is the tag
        if surface_form not in word_tag: 
            word_tag[surface_form] = tag 
            word_prob[surface_form] = prob

        # if we have seen the surface form before
        if surface_form in word_tag:
            # if the current probability is higher than the one we saw before
            if prob > word_prob[surface_form]:
                word_tag[surface_form] = tag
                word_prob[surface_form] = prob

    # read the input and assign the most frequent tag to each token

    input = open('sample_corpus.txt', 'r', encoding='utf-8')
    for line in input:
        line = line.strip() # strip off excess whitespace 
        # if there is no tab in the line, print it out and skip it	
        if '\t' not in line:
            print(line)
            continue
        line = line.split('\t')

        # if we have seen the surface form before (line[1]) then use the most probable tag
        if line[1] in word_tag:
            line[3] = word_tag[line[1]]
        # otherwise assign the most frequent tag to the input token
        else:
            line[3] = frequent_tag

        # print out the tagged line
        print('\t'.join(line))
        #print('%s\t_\t'% (line[1]) + '%s\t_\t_\t_\t_\t_\t' % (line[3]))

    fd.close()

Test()